In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb 

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
data_train = pd.read_csv("/kaggle/input/toxic-data-x2022gwt/train_data.csv")
data_test = pd.read_csv("/kaggle/input/toxic-data-x2022gwt/test_data.csv")

In [3]:
data_train

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,label,Assay_values
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1644
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2,2451
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,2,1384
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,16
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75378,11.460021,-3.868472,11.460021,0.053611,0.712426,230.245,220.165,230.036128,82.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,33
75379,5.928972,-2.841623,5.928972,0.082346,0.720533,313.747,296.611,313.041677,104.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1632
75380,4.975926,0.848333,4.975926,0.848333,0.596343,167.258,162.218,166.986341,50.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1373
75381,10.241948,0.324028,10.241948,0.324028,0.519485,128.215,112.087,128.120115,54.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [4]:
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

In [5]:
X = data_train.drop("label",axis=1)
y = data_train["label"]

In [6]:
clf1 = lgb.LGBMClassifier(random_state=40,n_estimators=2000,learning_rate=0.12)
clf4 = xgb.XGBClassifier(random_state=40,n_estimators=1500,eta=0.32)

In [7]:
voting_clf = VotingClassifier(estimators=[('lgm', clf1),  ('xgb', clf4)], voting='soft')

In [8]:
voting_clf.fit(X,y)

VotingClassifier(estimators=[('lgm',
                              LGBMClassifier(learning_rate=0.12,
                                             n_estimators=2000,
                                             random_state=40)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False, eta=0.32,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                             

In [9]:
predict = voting_clf.predict(data_test)

In [10]:
print(np.count_nonzero(predict==1))
print(np.count_nonzero(predict==2))

1317
9677


In [11]:
submission_file = pd.read_csv("/kaggle/input/the-toxicity-prediction-challenge-ii/sample_submission.csv")
submission_file.Predicted = predict

In [12]:
submission_file.to_csv("submissions.csv",index=False)